In [1]:
"""
Motor minimalista en Python similar a CROBOTS: versión didáctica con visualización en vivo y carga dinámica de bots.

Ejecutar: python crobots_py_starter.py

Características:
- Arena 2D simple con escenarios pseudoaleatorios (posiciones iniciales del robot + obstáculos)
- API de robot para estudiantes: implementar una clase que herede `RobotController` y anular `step(sensors)`
- Sensores: posición/rumbo propio, radar (objetivo más cercano dentro del alcance), estado de impacto
- Acciones: aceleración (-1..1), giro (-pi..pi), disparo() (un solo proyectil)
- Armas: proyectil simple con velocidad, alcance y colisión
- Bucle de simulación por turnos (dt fijo)
- Visualización con matplotlib (posiciones, rumbos, proyectiles)
- Carga dinámica de bots desde módulos externos de Python para facilitar su uso en el aula
"""

import importlib
import math
import random
import time
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List, Tuple, Optional, Dict

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt

In [3]:
# ---------- Constants ----------
ARENA_ANCHO = 800
ARENA_ALTO = 600
DT = 0.1
MAX_VELOCIDAD = 120.0
MAX_GIRO = math.pi
PROJECTILE_VELOCIDAD = 300.0
PROJECTILE_RANGO = 500.0    
RADAR_RANGO = 400.0
ROBOT_RADIO = 12
PROJECTILE_RADIO = 3
OBSTACULO_RADIO = 20

In [4]:
def clamp(x,a,b): return max(a, min(b, x))
def normalizar_angulo(a): return (a + math.pi) % (2*math.pi) - math.pi
def distancia(a,b): return math.hypot(a[0]-b[0],a[1]-b[1])

In [ ]:
@dataclass
class EstadoSensor:
    pos: Tuple[float, float]
    direccion: float
    distancia_radar: Optional[float]
    orientacion_radar: Optional[float]
    impacto: bool
    tiempo: float

class ControladorRobot:
    def __init__(self, nombre:str="robot"): self.name=nombre
    def init(self,info:Dict): self.info=info
    def step(self,sensors:EstadoSensor)->Dict: return {'aceleracion':0.0,'giro':0.0,'disparar':False}

class Projectil:
    def __init__(self, owner_id, x, y, direccion, tiempo_creacion):
        self.owner_id = owner_id
        self.x=x
        self.y=y
        self.direccion=direccion
        self.vx=math.cos(direccion)*PROJECTILE_VELOCIDAD
        self.vy=math.sin(direccion)*PROJECTILE_VELOCIDAD
        self.distancia_recorrida=0
        self.created_time=tiempo_creacion

    def step(self,dt):
        self.x+=self.vx*dt
        self.y+=self.vy*dt
        self.distancia_recorrida+=PROJECTILE_VELOCIDAD*dt

class Obstaculo:
    def __init__(self,x,y,r=OBSTACULO_RADIO): self.x=x; self.y=y; self.r=r

class EstadoRobot:
    def __init__(self, controlador, robot_id, x, y, direccion):
        self.controller=controlador
        self.id=robot_id
        self.x=x
        self.y=y
        self.heading=direccion
        self.vx=self.vy=0
        self.health=100
        self.hit=False
        self.last_fire_time=-1.0

class Arena:
    def __init__(self, width=ARENA_ANCHO, height=ARENA_ALTO, seed=None):
        self.width=width
        self.height=height
        self.seed=seed
        random.seed(seed)
        self.robots=[]
        self.projectiles=[]
        self.obstacles=[]
        self.time=0.0

    def add_robot(self, controlador, x, y, direccion):
        rid=len(self.robots)
        rc=EstadoRobot(controlador, rid, x, y, direccion)
        self.robots.append(rc)
        controlador.init({'arena_w':self.width, 'arena_h':self.height, 'robot_id':rid})
        return rc

    def spawn_random_scenario(self, n_robots=3, n_obstacles=6, margin=60, modulo_robot:Optional[List[str]]=None):
        self.obstacles=[Obstaculo(random.uniform(margin,self.width-margin),random.uniform(margin,self.height-margin)) for _ in range(n_obstacles)]
        self.robots=[]
        for i in range(n_robots):
            while True:
                x=random.uniform(margin,self.width-margin); y=random.uniform(margin,self.height-margin); rumbo=random.uniform(-math.pi,math.pi)
                if all(distancia((x,y), (obs.x,obs.y))>(obs.r+ROBOT_RADIO+10) for obs in self.obstacles): break
            controller=self._cargar_robot(modulo_robot[i]) if modulo_robot and i<len(modulo_robot) else RandomBot(f"bot_{i}")
            self.add_robot(controller, x, y, rumbo)

    def _cargar_robot(self, module_name:str) -> ControladorRobot:
        mod=importlib.import_module(module_name)
        for attr in dir(mod):
            obj=getattr(mod,attr)
            if isinstance(obj,type) and issubclass(obj,ControladorRobot) and obj is not ControladorRobot:
                return obj(attr)
        raise ValueError(f"No se encontro ninguna subclase RobotController en {module_name}")

    def _escanear(self,robot):
        distancia_mas_cercana = None
        rumbo_mas_cercano = None
        for otro in self.robots:
            if otro.id == robot.id: 
                continue
            d = distancia((robot.x, robot.y), (otro.x, otro.y))
            if d <= RADAR_RANGO and (distancia_mas_cercana is None or d < distancia_mas_cercana):
                bearing = normalizar_angulo(math.atan2(otro.y-robot.y, otro.x-robot.x)-robot.heading)
                distancia_mas_cercana, rumbo_mas_cercano = d, bearing
        return EstadoSensor((robot.x, robot.y), robot.heading, distancia_mas_cercana, rumbo_mas_cercano, robot.hit, self.time)

    def paso(self, dt=DT):
        acciones=[]
        for r in self.robots:
            sensors = self._escanear(r)
            try: acciones.append(r.controller.step(sensors))
            except: acciones.append({'throttle':0, 'turn':0, 'fire':False})

        for r, a in zip(self.robots, acciones):
            th = clamp(a.get('throttle',0),-1, 1)
            turn = clamp(a.get('turn',0),-MAX_GIRO*dt,MAX_GIRO*dt)
            r.heading = normalizar_angulo(r.heading+turn)
            speed = th*MAX_VELOCIDAD
            r.vx = math.cos(r.heading)*speed
            r.vy = math.sin(r.heading)*speed
            r.x = clamp(r.x+r.vx*dt,0,self.width)
            r.y = clamp(r.y+r.vy*dt,0,self.height)
            if a.get('fire', False) and self.time-r.last_fire_time>0.3:
                self.projectiles.append(Projectil(r.id, 
                                                  r.x+math.cos(r.heading)*(ROBOT_RADIO+2), 
                                                  r.y+math.sin(r.heading)*(ROBOT_RADIO+2), 
                                                  r.heading,
                                                  self.time))
                r.last_fire_time=self.time

        for p in list(self.projectiles):
            p.step(dt)
            if p.x<0 or p.x>self.width or p.y<0 or p.y>self.height or p.distance_travelled>PROJECTILE_RANGO:
                self.projectiles.remove(p)
                continue
            for r in self.robots:
                if r.id!=p.owner_id and distancia((p.x,p.y), (r.x,r.y)) < (PROJECTILE_RADIO+ROBOT_RADIO):
                    r.health-=25
                    r.hit=True
                    self.projectiles.remove(p)
                    break
        for r in self.robots:
            for obs in self.obstacles:
                d=distancia((r.x,r.y),(obs.x,obs.y))
                if d<(ROBOT_RADIO+obs.r):
                    dx = r.x-obs.x
                    dy = r.y-obs.y
                    nd = math.hypot(dx,dy) or 1e-3
                    overlap = (ROBOT_RADIO+obs.r)-nd
                    r.x += dx/nd*overlap
                    r.y += dy/nd*overlap
        for r in self.robots: 
            r.hit=False
        self.time+=dt

    def render_back(self, ax):
        ''' Esto esta muy mal pero anda mas o menos. Hace falta mejorar la visualizacion, MEJORES ROBOTS CON SKINS Y OBSTACULOS '''
        ax.clear()
        ax.cla()
        ax.set_xlim(0,self.width)
        ax.set_ylim(0,self.height)
        for obs in self.obstacles: 
            ax.add_patch(plt.Circle((obs.x, obs.y), obs.r, color='gray', alpha=0.4))
        for r in self.robots:
            ax.add_patch(plt.Circle((r.x, r.y), ROBOT_RADIO, color='blue' if r.health>0 else 'red', fill=True, alpha=0.5))
            ax.arrow(r.x, r.y, math.cos(r.heading)*15, math.sin(r.heading)*15, head_width=5, fc='k', ec='k')
        for p in self.projectiles:
            ax.add_patch(plt.Circle((p.x, p.y), PROJECTILE_RADIO, color='orange'))
        ax.set_title(f"t={self.time:.1f}s")

    def render(self, ax):
        """Render del estado actual del juego en el eje ax."""
        # Limpiar el contenido anterior
        ax.cla()

        # Configurar límites y aspecto
        ax.set_xlim(0, self.width)
        ax.set_ylim(0, self.height)
        ax.set_aspect('equal', adjustable='box')
        ax.set_xticks([])
        ax.set_yticks([])

        # Dibujar obstáculos
        for obs in self.obstacles:
            ax.add_patch(plt.Circle((obs.x, obs.y), obs.r, color='gray', alpha=0.4))

        # Dibujar robots
        for r in self.robots:
            color = 'blue' if r.health > 0 else 'red'
            ax.add_patch(plt.Circle((r.x, r.y), ROBOT_RADIO, color=color, alpha=0.6))
            ax.arrow(
                r.x, r.y,
                math.cos(r.heading) * 15,
                math.sin(r.heading) * 15,
                head_width=5, fc='k', ec='k'
            )

        # Dibujar proyectiles
        for p in self.projectiles:
            ax.add_patch(plt.Circle((p.x, p.y), PROJECTILE_RADIO, color='orange'))

        # Título con tiempo simulado
        ax.set_title(f"t = {self.time:.1f}s")

        # Refrescar el canvas (clave en Jupyter)
        ax.figure.canvas.draw_idle()
        ax.figure.canvas.flush_events()


In [6]:
class RandomBot(ControladorRobot):
    def paso(self,s):
        if random.random()<0.05: self._turn=random.uniform(-0.5,0.5)
        th=0.6
        turn=getattr(self,'_turn',0)*DT
        fire=s.radar_distance and s.radar_distance<200 and random.random()<0.25
        return {'throttle':th,'turn':turn,'fire':fire}

class SeekerBot(ControladorRobot):
    def paso(self,s):
        if s.radar_distance is None: 
            return {'throttle':0.2,'turn':0.1*DT,'fire':False}
        turn = clamp(s.radar_bearing*3.0, -MAX_GIRO*DT, MAX_GIRO*DT)
        fire = s.radar_distance < 220 and abs(s.radar_bearing) < 0.3
        return {'throttle':0.8,'turn':turn,'fire':fire}

In [7]:
def run_demo(n_robots=3, seed=None, ticks=300, bot_modules:Optional[List[str]]=None):
    arena = Arena(seed=seed)
    arena.spawn_random_scenario(n_robots = n_robots, n_obstacles = 6, modulo_robot = bot_modules)
    fig, ax = plt.subplots()
    plt.ion()
    plt.show()
    for t in range(ticks):
        arena.paso(DT)
        arena.render(ax)
        plt.pause(0.01)
        alive = [r for r in arena.robots if r.health > 0]
        if len(alive) <= 1: 
            break
    plt.ioff()
    plt.show()

In [8]:
# print("Running CROBOTS Python demo (visual mode)...")
# run_demo(n_robots=3,seed=42)
# print("Done.")

In [9]:
import numpy as np
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt

def save_simulation_gif(arena, filename="battle.gif", frames=300, dt=0.1, fps=20, figsize=(8,6)):
    """
    Ejecuta la simulación y guarda un GIF animado del resultado.
    Compatible con JupyterLab (usa backend 'Agg').
    """
    # Forzar backend no interactivo (para evitar errores con ipympl)
    matplotlib.use("Agg")

    pil_frames = []
    fig, ax = plt.subplots(figsize=figsize)

    for i in range(frames):
        # Avanzar la simulación
        arena.paso(dt)

        # Dibujar el frame actual
        ax.cla()
        arena.render(ax)

        # Dibujar en canvas
        fig.canvas.draw()

        # Convertir a array numpy desde buffer RGB
        image = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
        image = image.reshape(fig.canvas.get_width_height()[::-1] + (4,))
        image = Image.fromarray(image[..., :3])  # eliminar canal alfa

        pil_frames.append(image)

    plt.close(fig)

    # Guardar como GIF
    duration_ms = int(1000 / fps)
    pil_frames[0].save(
        filename,
        save_all=True,
        append_images=pil_frames[1:],
        duration=duration_ms,
        loop=0,
        optimize=False
    )

    print(f"GIF guardado correctamente en: {filename}")


In [10]:
arena = Arena(seed=42)
arena.spawn_random_scenario(n_robots=3, n_obstacles=5)
save_simulation_gif(arena, "mi_batalla.gif", frames=200, dt=0.1, fps=20)

GIF guardado correctamente en: mi_batalla.gif
